### SETUP INICIAL DO PROJETO

In [ ]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account


# carregar o .env com as credenciais
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")


# detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"


# parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.purchasing-vendor"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"


# configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


#doc: tables_to_process: lista de tabelas que serão processadas
#     output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned

In [ ]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

In [ ]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [ ]:
# Setup inicial do df para realizar a EDA 

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [ ]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")


print("Todas as tabelas foram processadas com sucesso!")

In [ ]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

In [ ]:
# Atribuir o df a uma variável com nome mais simples
purchasing_vendor = df_processados['purchasing_vendor']

print(f"Colunas: {purchasing_vendor.shape[1]}\nLinhas: {purchasing_vendor.shape[0]}")

In [ ]:
purchasing_vendor.head()

In [ ]:
# Obter os valores distintos da coluna
distinct_values = purchasing_vendor['purchasingwebserviceurl'].unique()

# Contar a quantidade de valores distintos
distinct_count = purchasing_vendor['purchasingwebserviceurl'].nunique()

# Exibir os resultados
print(f"Quantidade de valores distintos: {distinct_count}")
print("Valores distintos:")
print(distinct_values)


In [ ]:
# Remover a coluna 'purchasingwebserviceurl'
purchasing_vendor = purchasing_vendor.drop(columns=['purchasingwebserviceurl'])


In [ ]:
# Identificar duplicadas
duplicadas = purchasing_vendor[purchasing_vendor.duplicated(subset=['businessentityid'], keep=False)]

# Verificar se existem duplicadas
if not duplicadas.empty:
    
    duplicadas_ordenadas = duplicadas.sort_values(by=['businessentityid', 'modifieddate'])

    print("duplicadas ordenadas:")
    print(duplicadas_ordenadas)
else:
    print("Não foram encontradas duplicadas.")

In [ ]:
# Remover duplicadas* 
purchasing_vendor = purchasing_vendor.drop_duplicates(subset=['businessentityid'], keep='last')

print(f"Linhas após remover duplicadas (baseando-se na última 'modifieddate'): {len(purchasing_vendor)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicadas = purchasing_vendor.copy()


#doc: bkp = backup (cópia)
#doc*: mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
#      Importante, pois se houver erro na ingestão (duplicação), mantém os dados íntegros.

In [ ]:
# Ordenar e exibir o df por 'businessentityid'
purchasing_vendor = purchasing_vendor.sort_values(by=['businessentityid'])

print(purchasing_vendor)

In [ ]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in purchasing_vendor.columns:   
    missing_rows = purchasing_vendor[purchasing_vendor[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

In [ ]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = purchasing_vendor.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

In [ ]:
#verificar informações do df
purchasing_vendor.info()

In [ ]:
#avaliando as variáveis qualitativas*

coluna_quantitativa = ["preferredvendorstatus", "activeflag","creditrating"]
for col in coluna_quantitativa:
    counts = purchasing_vendor[col].value_counts().nlargest(10)
    percentages = (counts / purchasing_vendor.shape[0] * 100).map("{:.2f}%".format)
    summary = pd.DataFrame({"qtde.": counts, "%": percentages})
    print(summary)    


#doc*: variáveis qualitativas são um tipo de variável estatística que representam características ou atributos dos dados, sem serem medidas numericamente
#      no nosso caso, "preferredvendorstatus", "activeflag, por exemplo

#doc: a maioria dos fornecedores (quase 90%) é marcada como "preferidos", indicando que são priorizados no processo de compra.

#doc: a grande maioria dos fornecedores (96%) está ativa, sugerindo que o sistema mantém apenas um pequeno número de registros inativos.

#doc: a predominância de notas 1 (80.77%) indica que a empresa prioriza trabalhar com fornecedores de alta credibilidade, o que reduz riscos no processo de compras


In [ ]:
# Criar um gráfico de barras empilhadas
cross_tab = pd.crosstab(purchasing_vendor['preferredvendorstatus'], purchasing_vendor['activeflag'])
cross_tab.plot(kind='bar', stacked=True, figsize=(8, 6), color=['blue', 'orange'])

plt.title('Distribuição de ActiveFlag por PreferredVendorStatus', fontsize=14)
plt.xlabel('PreferredVendorStatus', fontsize=12)
plt.ylabel('Contagem', fontsize=12)
plt.legend(title='ActiveFlag', fontsize=10, title_fontsize=12)
plt.grid(False)   
plt.show()


#doc: status de fornecedor preferido (preferredvendorstatus) é distribuído entre os ativos e inativos.

In [ ]:
# Agrupar dados e plotar
preferred_credit = purchasing_vendor.groupby('preferredvendorstatus')['creditrating'].value_counts().unstack()
preferred_credit.plot(kind='barh', figsize=(10, 6), color=['purple', 'green', 'red'])

plt.title('Distribuição de CreditRating por PreferredVendorStatus', fontsize=14)
plt.xlabel('Contagem', fontsize=12)
plt.ylabel('PreferredVendorStatus', fontsize=12)
plt.legend(title='CreditRating', fontsize=10, title_fontsize=12)
plt.grid(False)  
plt.show()


#doc: a verificar como a avaliação de crédito é distribuída por status de fornecedor preferido.

In [ ]:
purchasing_vendor.info()

In [ ]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['purchasing_vendor'] = purchasing_vendor

In [ ]:
# Padronizar colunas com valores textuais
purchasing_vendor['name'] = purchasing_vendor['name'].str.strip().str.upper()
purchasing_vendor['accountnumber'] = purchasing_vendor['accountnumber'].str.strip().str.upper()

print(purchasing_vendor.head())

#doc: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

In [ ]:
# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
 
    output_table = f"{output_dataset}.{table_name}"
   
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  
    )
    
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")